<a href="https://colab.research.google.com/github/Narendra-Sompalle/GenAIProjects/blob/main/Mistral_Langchain_Llamaindex_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Webviate_API_Key="vytPtirZCjyYQNHT6llKIv2gRFhqg9CJnfyG"
webviate_end_point="grpc-my8qpt4twabk4h6exrra.c0.asia-southeast1.gcp.weaviate.cloud"

In [ ]:
! pip install langchain tiktoken pypdf
! pip install langchain_community

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
documents=PyPDFLoader("/content/Fine-tune_the_Entire_RAG_Architecture_including_DP.pdf")

In [ ]:
docs=documents.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [ ]:
all_docs=splitter.split_documents(docs)

In [ ]:
## storing documents into vectordb

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-11-0ce1537519dd>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

In [ ]:
weaviate_url="my8qpt4twabk4h6exrra.c0.asia-southeast1.gcp.weaviate.cloud"
weaviate_api_key="vytPtirZCjyYQNHT6llKIv2gRFhqg9CJnfyG"

In [ ]:
# ! pip install weaviate-client

In [ ]:
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.classes.init import Auth


# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

In [ ]:
client

In [ ]:
class_obj = {
    "class": "Document",
    "vectorizer": "none",  # We provide vectors manually
    "properties": [
        {
            "name": "text",
            "dataType": ["text"]
        }
    ]
}


In [ ]:
# dir(client)

In [ ]:
Weaviate.from_documents(client=client,embedding=embeddings,documents=all_docs)

AttributeError: 'WeaviateClient' object has no attribute 'schema'

## LLmaindex local


In [2]:
%pip install llama-index-llms-huggingface

In [3]:
!pip install llama-index

In [4]:
!mkdir data


mkdir: cannot create directory ‘data’: File exists


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [6]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()

In [7]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [8]:
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
          prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
          prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
          prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt
def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


In [9]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    model_name="HuggingFaceH4/zephyr-7b-beta",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model =HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [12]:
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
Settings.llm=llm
Settings.embed_model=embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

# ... until you create a query engine
query_engine = index.as_query_engine(llm=llm)